In [24]:
# Dependencies
import matplotlib.pyplot as plt
import requests
from scipy import stats
import pandas as pd
from config import api_key

In [25]:
# Save Config information
url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY"
size = "&outputsize=full"
key = f"&apikey={api_key}"

query_url = f"{url}&symbol="

In [6]:
# List of desired ticker symbols
tickers = ["META", "AAPL", "AMZN", "NFLX", "GOOG"]

# Row to store data
rows = []

# Loop through tickers symbols, making an API call for each
for ticker in tickers:
    response = requests.get(query_url + ticker + size + key).json()
    # Loop through each ticker symbol's response, storing response data to dictionary
    for date, values in response["Time Series (Daily)"].items():
        symbol = response['Meta Data']['2. Symbol']
        row = {
            'symbol' : symbol,
            'date': date,
            'open': float(values['1. open']),
            'high': float(values['2. high']),
            'low': float(values['3. low']),
            'close': float(values['4. close']),
            'volume': int(values['5. volume'])
        }
        rows.append(row)

# Create DataFrame from the list of dictionaries
pd.set_option('display.max_rows', None)
df = pd.DataFrame(rows)

# Set 'symbol' column as the index
df.set_index('symbol', inplace=True)

df.head()

,date,open,high,low,close,volume
symbol,,,,,,
META,2024-02-08,468.32,470.59,465.0300,470.00,18815097
META,2024-02-07,458.00,471.52,456.1773,469.59,23065994
META,2024-02-06,464.00,467.12,453.0000,454.72,21655214
META,2024-02-05,469.88,471.90,459.2200,459.41,40832376
META,2024-02-02,459.60,485.96,453.0100,474.99,84707646


In [14]:
# Facebook Stock data
meta_df = df.loc["META"] 
meta_df.head()

,date,open,high,low,close,volume
symbol,,,,,,
META,2024-02-08,468.32,470.59,465.0300,470.00,18815097
META,2024-02-07,458.00,471.52,456.1773,469.59,23065994
META,2024-02-06,464.00,467.12,453.0000,454.72,21655214
META,2024-02-05,469.88,471.90,459.2200,459.41,40832376
META,2024-02-02,459.60,485.96,453.0100,474.99,84707646


In [15]:
# Apple Stock data 
aapl_df = df.loc["AAPL"]
aapl_df.head()

,date,open,high,low,close,volume
symbol,,,,,,
AAPL,2024-02-08,189.385,189.535,187.3500,188.32,40962046
AAPL,2024-02-07,190.640,191.050,188.6100,189.41,53438955
AAPL,2024-02-06,186.860,189.310,186.7695,189.30,43490759
AAPL,2024-02-05,188.150,189.250,185.8400,187.68,69668820
AAPL,2024-02-02,179.860,187.330,179.2500,185.85,102551680


In [19]:
# Amazon Stock data
amzn_df = df.loc["AMZN"]
amzn_df.head()

,date,open,high,low,close,volume
symbol,,,,,,
AMZN,2024-02-08,169.65,171.43,168.88,169.84,42316454
AMZN,2024-02-07,169.48,170.88,168.94,170.53,47174060
AMZN,2024-02-06,169.39,170.71,167.65,169.15,42505518
AMZN,2024-02-05,170.20,170.55,167.70,170.31,55081297
AMZN,2024-02-02,169.19,172.50,167.33,171.81,117218313


In [20]:
# Netflix Stock data
nflx_df = df.loc['NFLX']

nflx_df.head()

,date,open,high,low,close,volume
symbol,,,,,,
NFLX,2024-02-08,560.55,563.7000,555.740,558.53,3175468
NFLX,2024-02-07,558.16,567.8000,554.980,559.30,4373591
NFLX,2024-02-06,564.12,566.3399,554.370,555.88,2840300
NFLX,2024-02-05,562.47,575.7500,557.195,562.06,4143099
NFLX,2024-02-02,564.52,567.8000,562.370,564.64,4032843


In [22]:
# Google Stock data
goog_df = df.loc["GOOG"]

goog_df.head()

,date,open,high,low,close,volume
symbol,,,,,,
GOOG,2024-02-08,146.97,147.61,146.4200,147.22,18241319
GOOG,2024-02-07,146.12,147.00,145.2103,146.68,21436126
GOOG,2024-02-06,145.96,146.74,144.5200,145.41,21517655
GOOG,2024-02-05,144.04,146.67,143.9100,144.93,29254444
GOOG,2024-02-02,140.89,143.88,138.1700,143.54,42136127


In [30]:
# Import dependencies 
import psycopg2
from sqlalchemy import create_engine

In [36]:
# Postgres SQL connection information
dbname = 'stock_db'
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432


In [40]:
# Establish connection 
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')


In [41]:
# Define dataframe and table names
dataframes = [meta_df, aapl_df, amzn_df, nflx_df, goog_df]
table_names = ['meta', 'aple', 'amazon', 'netflix', 'google']

# Loop through the DataFrame and table names
for df, table_name in zip(dataframes, table_names): 
    # Write DataFrame to PostgresSQL 
    df.to_sql(table_name, engine, if_exists='replace', index=False)

conn.commit()
conn.close()
